### Import required dependencies

In [1]:
from arcgis.gis import *
from IPython.display import display
from getpass import getpass

### Set the information to connect to your portal / gis

In [3]:
url = 'https://esrica-ncr.maps.arcgis.com'
user_name = 'fancelin_ncr'
user_password = getpass()

········


### Connect to your portal and display its information

In [4]:
gis = GIS(url=url, username=user_name, password=user_password)
gis

GIS @ https://esrica-ncr.maps.arcgis.com version:8.1

### Retrieve the current user object

In [5]:
user = gis.users.me
user

<User username:fancelin_ncr>

## Say, I want to know all the user in my portal

### The way it works according to documentation.

#### The regular search

https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.search

`search(query=None, sort_field='username', sort_order='asc', max_users=100, outside_org=False, exclude_system=False, user_type=None, role=None)`

| Argument | Description |
| --------- | ------------- |
|query|Required String. The search query.|
|sort_field|Optional string. Valid values can be username (the default) or created.|
|sort_order|Optional string. Valid values are asc (the default) or desc.|
|max_users|Optional integer. The maximum number of users to be returned. The default is 100.|
|outside_org|Optional boolean. This controls whether to search outside your organization. The default is False|
|exclude_system|Optional boolean. Controls if built-in system accounts are returned or not. True means built-in account are not returned, where as False means that they are.|
|user_type|	Optional String. This parameters allows for the filtering of the users by their assigned typ|
|role|Optional String. Specify the roleId. This parameter allows for the filting of the users based on a roleId.|



#### The advanced search

https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.advanced_search

`advanced_search(query, return_count=False, max_users=10, start=1, sort_field='username', sort_order='asc', as_dict=False)`


| Argument | Description |
| --------- | ------------- |
|query|Required String. The search query.|
|return_count|Optional Boolean. If True, the number of users found by the query string is returned.
|max_users|Optional Integer. Limits the total number of users returned in a a query. The default is 10 users. If all users is needed, -1 should be used..|
|start|Optional Int. The starting position to search from. This is only required if paging is needed.
|sort_field|Optional String. Responses from the search operation can be sorted on various fields. avgrating is the default.|
|sort_order|Optional String. The sequence into which a collection of records are arranged after they have been sorted. The allowed values are: asc for ascending and desc for descending.|
|as_dict|Required Boolean. If True, the response comes back as a dictionary.


#### How to use the advanced search

In [ ]:
# How to get your organization id https://esrica-ncr.maps.arcgis.com/sharing/rest/portals/self?culture=en&f=pjson
users = gis.users.advanced_search(
    query='orgid:vY6WuhLW0HkFe6Fl', 
    return_count=False, 
    max_users=100, 
    start=1, 
    sort_field='username', 
    sort_order='asc', 
    as_dict=False
)
print('Number of users with an advanced search: {}'.format(len(users['results'])))
users['results']

 You can get your organization id from a rest endpoint. Here is an example for our NCR office:
 
 `https://esrica-ncr.maps.arcgis.com/sharing/rest/portals/self?culture=en&f=pjson`
 
 
 ```json
 "homePageFeaturedContent": "id:72d70011859642819e2d84280056eb8f",
  "homePageFeaturedContentCount": 12,
  "id": "vY6WuhLW0HkFe6Fl", // This is your organization id
  "isPortal": false,
  "layerTemplatesGroupQuery": "title:\"Esri Layer Templates\" AND owner:esri_en",
  "livingAtlasGroupQuery": "title:\"LAW Search\" AND owner:Esri_LivingAtlas",
  "metadataEditable": true,
  "metadataFormats": [
    "iso19139"
  ],
 ```

Is there a catch? Of course there is: <font color=red>advanced_search does not work at 1.7, which ships with ArcGIS Pro 2.5</font> You will need Python API for ArcGIS version 1.8 or above.

### So does it means the regular search does not support returning all users?

Correct! Unlike advanced search, if you pass -1 as a max_users parameter value, you will get a weird behaviour. It will returns only 10 user. <font color=red>This is not even the value for max_users</font>

In [35]:
users = gis.users.search(max_users=-1)
for user in users:
    print(user.username + "\t:\t" + str(user.role))
print('Total Number of User: {}'.format(len(users)))

abaker_esrica_ncr	:	org_admin
abodnar_ncr	:	org_admin
afrost_ncr9	:	org_publisher
bladdsncr	:	org_admin
cgorman_esrica_ncr	:	org_admin
clawlor	:	org_admin
cmaccormack_esrica_ncr	:	org_admin
dgreen_NCR	:	org_publisher
dprice__ncr	:	org_admin
echiasson_ncr	:	org_admin
Total Number of User: 10


### So is there a good workaround?




You can use the number of license being used.
https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager.counts


In [8]:
licenses = gis.users.counts(type='user_type', as_df=False,)
licenses

[{'key': 'fieldWorkerUT', 'count': 1},
 {'key': 'GISProfessionalAdvUT', 'count': 26},
 {'key': 'GISProfessionalBasicUT', 'count': 3},
 {'key': 'GISProfessionalStdUT', 'count': 1},
 {'key': 'storytellerUT', 'count': 2}]

Then sum the information obtained from there.

In [9]:
users_count = sum([value['count'] for value in licenses])
users_count

33

That matches your number of matched used

In [31]:
users = gis.users.search(max_users=150, exclude_system=True)
print('Number of users with a regular search: {}'.format(len(users)))

Number of users with a regular search: 33


####  What if you want to abolutely get all the users information

If you want to guarantee that your retriving all the users, you can make sure you check that the maximum number of users returned is never equals to the maximum user you specified.

In [39]:
def get_users(max_users=5):
    users = gis.users.search(max_users=max_users)
    if len(users) == max_users:
        return get_users(max_users=max_users * 2)
    return users

users = get_users(max_users=5)
for user in users:
    print(user.username + "\t:\t" + str(user.role))
print('Total Number of User: {}'.format(len(users)))

abaker_esrica_ncr	:	org_admin
abodnar_ncr	:	org_admin
afrost_ncr9	:	org_publisher
bladdsncr	:	org_admin
clawlor	:	org_admin
cmaccormack_esrica_ncr	:	org_admin
dgreen_NCR	:	org_publisher
dprice__ncr	:	org_admin
echiasson_ncr	:	org_admin
esrica-ncr	:	org_admin
fancelin_ncr	:	org_admin
ghunter_ncr9	:	org_admin
gplunkett_esrica_ncr	:	org_admin
jbart_esrica_ncr	:	org_admin
jcormier_ncr	:	org_admin
jhughes_NCR4	:	org_admin
jingraham_ncr	:	org_admin
jting_ncr	:	org_admin
khibma_ncr	:	org_admin
kravnas_NCR	:	org_publisher
lmccallum_esrica_ncr	:	org_admin
mgallant_ncr	:	org_admin
mguo_ncr	:	org_admin
mschmidt_ncr	:	org_admin
msdi_test	:	iAAAAAAAAAAAAAAA
mshao_ncr	:	org_admin
NCR_Demo	:	iAAAAAAAAAAAAAAA
pbilodeau_NCR5	:	org_publisher
rvirk_ncr	:	org_admin
shotte_ncr	:	org_admin
sseymour1	:	org_publisher
twilson_ncr	:	org_admin
WorldReach_ncr	:	t0Jx3qpYQxkJF1Kg
Total Number of User: 33
